# 주식등락예측하기 (머신러닝) 4.29

단순히 과거의 데이터들로 학습을 하여 그날의 데이터로 다음날의 주가는 상승할 것인지 하락할 것인지를 맞추는 경우입니다.

## 1. 네이버 금융에서 주식데이터 가져오기

In [1]:
# 기본적으로 필요한 라이브러리
import numpy as np
import pandas as pd

import timeit #데이터를 가져오는 데 얼마나 시간이 걸리는지 확인하기 위한 라이브러리

### 1.1 원하는 회사의 상장코드 가져오기

In [2]:
#KRX 한국 거래소에 상장되어있는 회사들의 데이터를 가져온다.
#홈페이지에서 바로 가져오므로, 데이터가 최신일것임.
listing = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]

#종목코드는 6자리이므로 6자리로 맞춰준다.
#필요한 것은 회사명과 종목코드이기 때문에 필요 없는 column들은 제외해준다.
listing['종목코드'] = listing['종목코드'].map('{:06d}'.format)
listing = listing[['회사명', '종목코드']]

#한글로된 컬럼명을 영어로 바꿔준다.
#외부라이브러리들 때문에 왠만하면 한글 데이터는 영어로 바꿔주는 것이 좋음.
listing = listing.rename(columns={'회사명':'name', '종목코드':'code'})

# 원하는 종목의 코드를 가져오는 함수
def get_url(item_name, data):
    code = data[data['name'] == item_name]['code']
    url = ('http://finance.naver.com/item/sise_day.nhn?code=' + code.iloc[0])
    return url

company = '진에어'
url = get_url(company, listing)
print('{} stock url : {}'.format(company, url))

진에어 stock url : http://finance.naver.com/item/sise_day.nhn?code=272450


### 1.2 원하는 페이지까지 데이터 가져오기

In [3]:
#일자 데이터를 담을 stock이라는 DataFrame 정의
stock = pd.DataFrame()

# 1페이지에서 원하는 페이지까지 가져오기
# 1페이지당 10개의 데이터가 존재
import timeit
start = timeit.default_timer() # 시간이 얼마나 걸리는지 시작.

for page in range(1, 401):
    pg_url = '{url}&page={page}'.format(url=url, page=page)
    stock = stock.append(pd.read_html(pg_url, header=0)[0], ignore_index=True)

stock = stock.dropna()

stop = timeit.default_timer()
print(stop - start)

63.755416334808565


In [4]:
stock_data = stock.sort_values(by=['날짜']) #오래된 순으로 다시 재정렬
stock_data.head()

,날짜,종가,전일비,시가,고가,저가,거래량
5997,2017.12.08,28850.0,200.0,28650.0,29600.0,26000.0,7793658.0
1737,2017.12.08,28850.0,200.0,28650.0,29600.0,26000.0,7793658.0
1752,2017.12.08,28850.0,200.0,28650.0,29600.0,26000.0,7793658.0
1767,2017.12.08,28850.0,200.0,28650.0,29600.0,26000.0,7793658.0
1782,2017.12.08,28850.0,200.0,28650.0,29600.0,26000.0,7793658.0


## 2. 머신러닝 준비하기

데이터의 칼럼을 영어로 바꿔준다.

In [5]:
stock_data.columns = ['date', 'close', 'diff' , 'start', 'high' ,'low', 'volume']
stock_data = stock_data.set_index('date')

등락을 예측하기위해 등락 칼럼을 만들어 준다.

In [6]:
stock_data.describe()

,close,diff,start,high,low,volume
count,3633.000000,3633.000000,3633.000000,3633.000000,3633.000000,3.633000e+03
mean,26796.917148,591.866226,27009.234792,27453.660886,26134.227911,1.288864e+06
std,1745.072553,612.377414,1773.700250,1826.539791,1582.321739,2.232102e+06
min,17000.000000,0.000000,16500.000000,17450.000000,16500.000000,3.910100e+04
25%,26300.000000,300.000000,26400.000000,27000.000000,26000.000000,2.107500e+05
50%,26850.000000,350.000000,27000.000000,27300.000000,26150.000000,5.228570e+05
75%,27150.000000,550.000000,27550.000000,27700.000000,26600.000000,8.373450e+05
max,33800.000000,3000.000000,33850.000000,34300.000000,32650.000000,7.793658e+06


In [7]:
def up_down(x):
    if x >= 0:
        return 1
    else :
        return 0
stock_data['diff'] = (stock_data['close'].shift(-1)-stock_data['close']).apply(up_down)

stock_data.head()

,close,diff,start,high,low,volume
date,,,,,,
2017.12.08,28850.0,1,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,1,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,1,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,1,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,1,28650.0,29600.0,26000.0,7793658.0


최종적으로 성능을 확인하기 위해 7:3 비율로 train데이터와 test데이터를 만들어 준다.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
y = stock_data['diff'].iloc[:-1] # target인 dependent variable
X = stock_data.drop('diff', axis=1) # independent variable
today = X.iloc[-1]
X = X.iloc[:-1]
#주식데이터는 시간도 영향이 있어야 할 거 같은데.. 이문제는 딥러닝 LSTM밖에 생각이 안나네요. 일단 ㄱㄱ
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, shuffle=False )

In [10]:
X_train.head()

,close,start,high,low,volume
date,,,,,
2017.12.08,28850.0,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,28650.0,29600.0,26000.0,7793658.0
2017.12.08,28850.0,28650.0,29600.0,26000.0,7793658.0


In [11]:
X_test.head()

,close,start,high,low,volume
date,,,,,
2017.12.18,26850.0,27250.0,27300.0,26550.0,210750.0
2017.12.18,26850.0,27250.0,27300.0,26550.0,210750.0
2017.12.18,26850.0,27250.0,27300.0,26550.0,210750.0
2017.12.18,26850.0,27250.0,27300.0,26550.0,210750.0
2017.12.18,26850.0,27250.0,27300.0,26550.0,210750.0


In [12]:
y_train.head()

date
2017.12.08    1
2017.12.08    1
2017.12.08    1
2017.12.08    1
2017.12.08    1
Name: diff, dtype: int64

In [13]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# end start 데이터들의 왜도와 첨도 확인 및 히스토그램 그리기
# f, ax = plt.subplots(1,2, figsize=(15,6))

# columns = ['end', 'start']

# count = 0
# for row in range(2):
#     sns.distplot(X_train[columns[count]], ax=ax[row])
#     print("%s -> Skewness: %f, Kurtosis: %f" %  (columns[count],X_train[columns[count]].skew(), 
#                                                      X_train[columns[count]].kurt()))
#     count += 1

In [14]:
# end start 데이터들의 왜도와 첨도 확인 및 히스토그램 그리기
# f, ax = plt.subplots(1,3, figsize=(15,6))

# columns = ['high','low','volume']

# count = 0
# for row in range(3):
#     sns.distplot(X_train[columns[count]], ax=ax[row])
#     print("%s -> Skewness: %f, Kurtosis: %f" %  (columns[count],X_train[columns[count]].skew(), 
#                                                      X_train[columns[count]].kurt()))
#     count += 1

# 3. 모델링

In [15]:
# Importing Classifier Modules
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

D:\Users\WIN10\Anaconda3\envs\python3.5\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [16]:
#cross_validation
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#cross_val_score에서 분류모형의 scoring은 accuracy이다.
kfold = KFold(n_splits = 5, shuffle = True, random_state = 42)

Logistic = LogisticRegression()
KNN = KNeighborsClassifier()
DeicisonTree = DecisionTreeClassifier()
RandomForest = RandomForestClassifier()
NaiveBayes = GaussianNB()
SVM = SVC()

models = [{'name' : 'Logistic', 'model' : Logistic}, {'name' : 'KNN', 'model' : KNN},
        {'name' : 'DeicisonTree', 'model' : DeicisonTree}, {'name' : 'RandomForest', 'model' : RandomForest},
        {'name' : 'NaiveBayes', 'model' : NaiveBayes}, {'name' : 'SVM', 'model' : SVM}]

def cv_accuracy(models):
    for m in models:
        print("Model {} CV score : {:.4f}".format(m['name'], 
                                                  np.mean(cross_val_score(m['model'], 
                                                                          X_train, y_train, cv=kfold))))
    

In [17]:
cv_accuracy(models)

Model Logistic CV score : 0.9984
Model KNN CV score : 0.9984
Model DeicisonTree CV score : 0.9984
Model RandomForest CV score : 0.9984
Model NaiveBayes CV score : 0.9155
Model SVM CV score : 0.9984


LogisticRegression이 제일 성능이 좋으므로 최종 모델로 선택

In [18]:
from sklearn import datasets
from sklearn import metrics

model = Logistic
model.fit(X_train, y_train)
print (model)

predicted = model.predict(X_test)

#Accuracy : 전체 샘플 중 맞게 예측한 샘플수의 비율
#Precision(정밀도) : postive라고 예측한 것 중에서 실제 postive인 것
#Recall(재현율) : 실제 postive중에 예측한 postive 비율
print (metrics.classification_report(y_test, predicted))

#confusion_matrix에서
#행은 실제값, 열은 예측한 값으로 0 1 순서대로 임
print (metrics.confusion_matrix (y_test, predicted))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
             precision    recall  f1-score   support

          0       1.00      0.01      0.01       169
          1       0.85      1.00      0.92       921

avg / total       0.87      0.85      0.78      1090

[[  1 168]
 [  0 921]]


# 문제점
---------version1 04/22--------------
* 주가는 시간에 따라 변화도 생각해봐야함. 
* 기술적 분석뿐만 아닌 다른 영향들에 대한 feature도 도입해보아야함.
* 머신러닝은 예측할 때 학습한 feature도 똑같이필요하므로 지금의 데이터는 하루가 끝이나야 얻을 수 있는 데이터로 미래를 예측하기에 적합하지 않은 feature들이다.(ver2에서 수정)
* 전날의 종가, 시가, 거래량, 고가, 저가로 오늘의 시가 또는 종가를 예측할 수 있을까라는 의문 (ver2에서 수정)

---------version2 04/29--------------
* train과 test데이터를 나눌때 과연 랜덤으로 나누는 것이 더 좋을 지 시간의 순서에 맞게 셔플하지 않고 나누는 것이 더 좋을지 확인
* 현재 오늘의 시가, 종가, 거개량, 고가, 저가로 내일의 종가는 오를것인지 내릴것인지 예측하는 모델로 구현
* 성능이 50%로 그냥 싹다 1로 한거임.

# 오늘의 종가 예측 (오를 것인가? 내릴 것인가?)

In [19]:
#예측한 모델의 마지막값 도출 -> 즉 어제의 데이터로인해 내일의 종가가 오를 것인지 내릴 것인지?
predicted[-15:]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [20]:
len(predicted)

1090

종가가 오를 것이라고 예측

In [21]:
#오늘데이터
today

close      24700.0
start      24550.0
high       25150.0
low        24450.0
volume    136974.0
Name: 2019.04.30, dtype: float64

In [22]:
#전날의 데이터
X_test.iloc[-1]

close      24500.0
start      24800.0
high       25600.0
low        24400.0
volume    261269.0
Name: 2019.04.29, dtype: float64

주가가 상승했음을 예측하였음

In [23]:
model.predict(pd.DataFrame(today).T)

array([1], dtype=int64)

# 그래프 그리기

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns

In [25]:
predict_data = pd.concat([X_test.reset_index(), pd.DataFrame(predicted, columns=['prediction'])], axis=1)
predict_data = predict_data.set_index('date')

In [ ]:
test_data = pd.concat([X_test, pd.DataFrame(y_test)], axis=1)
test_data.head()

In [ ]:
test_data.head()